In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Conv2D, MaxPooling2D, Flatten, Dense
%matplotlib inline 

Using TensorFlow backend.
/home/calvus/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/calvus/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/calvus/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/calvus/.local/lib/python3.7/site-packages/tensorflow/python

In [2]:
input_shape = (250, 250, 3)
batch = 8
lr = 0.002

In [3]:
img_gen = ImageDataGenerator(rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            rescale=1/255,
                            fill_mode='nearest')

In [4]:
train_gen = img_gen.flow_from_directory('dataset/train',
                                   target_size=input_shape[:2],
                                   batch_size=batch,
                                   class_mode='categorical',
                                   shuffle=True,
                                   seed=42)
test_gen = img_gen.flow_from_directory('dataset/test',
                                        target_size=input_shape[:2],
                                        batch_size=batch,
                                        class_mode='categorical',
                                        shuffle=True,
                                        seed=42)

Found 9867 images belonging to 3 classes.
Found 5247 images belonging to 3 classes.


In [5]:
train_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

In [6]:
test_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

In [13]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Conv2D(filters=64,kernel_size=(2,2), input_shape=input_shape, activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(filters=64,kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(4,4), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('sigmoid'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae', 'acc'])

In [14]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 248, 248, 32)      896       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 122, 122, 64)      18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 38, 38, 64)        36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 64)       

In [15]:
results = model.fit_generator(train_gen, epochs = 50, steps_per_epoch=150, 
                             validation_data=test_gen, validation_steps=10,
                             use_multiprocessing=True)

Epoch 1/50
150/150 [==============================] - 70s 468ms/step - loss: 0.1865 - mae: 0.3151 - acc: 0.5849 - val_loss: 0.1596 - val_mae: 0.3152 - val_acc: 0.6125
Epoch 2/50
150/150 [==============================] - 69s 461ms/step - loss: 0.1644 - mae: 0.2851 - acc: 0.6467 - val_loss: 0.0882 - val_mae: 0.2717 - val_acc: 0.5625
Epoch 3/50
150/150 [==============================] - 69s 460ms/step - loss: 0.1046 - mae: 0.1876 - acc: 0.7867 - val_loss: 0.0389 - val_mae: 0.1578 - val_acc: 0.8250
Epoch 4/50
150/150 [==============================] - 73s 486ms/step - loss: 0.1076 - mae: 0.1948 - acc: 0.7775 - val_loss: 0.1343 - val_mae: 0.1702 - val_acc: 0.8125
Epoch 5/50
150/150 [==============================] - 76s 510ms/step - loss: 0.0906 - mae: 0.1680 - acc: 0.8133 - val_loss: 0.0121 - val_mae: 0.1672 - val_acc: 0.8250
Epoch 6/50
150/150 [==============================] - 69s 460ms/step - loss: 0.1031 - mae: 0.1891 - acc: 0.7958 - val_loss: 0.0921 - val_mae: 0.1775 - val_acc: 0.825

Epoch 50/50
150/150 [==============================] - 68s 452ms/step - loss: 0.0640 - mae: 0.1190 - acc: 0.8767 - val_loss: 0.0607 - val_mae: 0.1523 - val_acc: 0.8125


In [17]:
score = model.evaluate_generator(test_gen, verbose=0)

In [18]:
print(score)

[0.005610339809209108, 0.1178390309214592, 0.8787878751754761]


In [19]:
model.save('First_try.h5')